In [5]:
# Librerías a utilizar
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
from openpyxl import load_workbook, Workbook
from openpyxl.styles import Font, PatternFill, Alignment
from openpyxl.utils import get_column_letter
from datetime import datetime

In [ ]:
# Preparación de rangos
def letra_a_indice_excel(letra: str) -> int:
    letra = letra.strip().upper()
    res = 0
    for ch in letra:
        if not ('A' <= ch <= 'Z'):
            raise ValueError("Letra inválida.")
        res = res * 26 + (ord(ch) - ord('A') + 1)
    return res - 1

def indice_a_letra_excel(ind: int) -> str:
    ind += 1
    s = ""
    while ind:
        ind, r = divmod(ind - 1, 26)
        s = chr(65 + r) + s
    return s

def safe_sheet_name(name: str) -> str:
    invalid = set(':\\/?*[]')
    cleaned = ''.join(ch for ch in str(name) if ch not in invalid)
    return cleaned[:31] if cleaned else "Hoja"

class ComparadorExcel:
    def __init__(self, root):
        self.root = root
        self.root.title("Comparador de Excel por Concepto")
        self.root.geometry("1150x850")
        self.root.minsize(1000, 700)
        self.root.configure(bg="#f8f8f8")

        # Estado
        self.path_prev = tk.StringVar()
        self.path_nuevo = tk.StringVar()
        self.path_base = tk.StringVar()  # Año anterior (base de referencia)
        self.hoja_combo = None
        self.scrollable_frame = None
        self.sheet = None

        # Parámetros de estructura
        self.col_concepto_letra = tk.StringVar(value="B")     # columna con nombres de conceptos (p.ej. meses)
        self.fila_concepto_ini = tk.StringVar(value="6")      # rango de filas de conceptos (p.ej. Enero..Diciembre)
        self.fila_concepto_fin = tk.StringVar(value="54")
        self.fila_meses = tk.StringVar(value="9")             # fila de cabecera (años en tu archivo)
        self.col_meses_ini = tk.StringVar(value="D")          # rango de columnas de cabecera (años)
        self.col_meses_fin = tk.StringVar(value="O")
        self.anio_entry = tk.StringVar(value="2025")

        # Detectados
        self.conceptos_lista = []         # lista de conceptos detectados (p.ej. meses)
        self.conceptos_check_vars = {}    # checkboxes para lista actual (conceptos o años)
        self.meses_orden = []             # cabeceras detectadas (años/meses)
        self.max_conceptos = 12           # límite de selección

        # Orientación de salida
        self.orientacion = tk.StringVar(value="Analisis_horizontal")

        # Lista donde guardaremos las pestañas que el usuario seleccione
        self.pestanas_seleccionadas = []

        self._crear_interfaz()
    # ------------------------ Utilidades ------------------------
   
    def _leer_valor_cabecera(self, cell, fallback):
        """Devuelve int si la cabecera es numérica, string si es texto, o fallback si None."""
        if cell is None:
            return fallback
        if isinstance(cell, (int, float)):
            return int(cell) if float(cell).is_integer() else cell
        try:
            return int(str(cell).strip())
        except ValueError:
            return str(cell).strip()

    # ------------------------ Interfaz ------------------------

    def _crear_interfaz(self):
        style = ttk.Style()
        style.configure("TLabel", font=("Segoe UI", 10))
        style.configure("TButton", font=("Segoe UI", 10), padding=6)
        style.configure("TCheckbutton", font=("Segoe UI", 10))
        style.configure("TLabelframe", font=("Segoe UI", 11, "bold"))
        style.configure("TLabelframe.Label", foreground="#1f4e79")

        # Barra superior
        frame_top = tk.Frame(self.root, bg="#f0f0f0")
        frame_top.pack(fill="x", padx=10, pady=5)

        tk.Label(frame_top, text="Comparador de Excel en serie de tiempo",
                 font=("Segoe UI", 14, "bold"), bg="#f0f0f0").pack(side="left", padx=10)

        tk.Button(frame_top, text="Comparar y exportar",
                  command=self._comparar, bg="#1f7a1f", fg="white",
                  font=("Segoe UI", 11, "bold")).pack(side="right", padx=10)

        ttk.Separator(self.root, orient="horizontal").pack(fill="x", padx=10, pady=5)

        # Archivos
        frame_archivos = tk.LabelFrame(self.root, text="Archivos Excel", padx=10, pady=10)
        frame_archivos.pack(fill="x", padx=10, pady=10)

        # Archivo año anterior (base)
        tk.Label(frame_archivos, text="Archivo año anterior:").grid(row=0, column=0, sticky="w")
        tk.Entry(frame_archivos, textvariable=self.path_base, width=80).grid(row=0, column=1, padx=5)
        ttk.Button(frame_archivos, text="Seleccionar", command=lambda: self._seleccionar_archivo("base")).grid(row=0, column=2, padx=5)

        # Archivo previo
        tk.Label(frame_archivos, text="Archivo previo:").grid(row=1, column=0, sticky="w")
        tk.Entry(frame_archivos, textvariable=self.path_prev, width=80).grid(row=1, column=1, padx=5)
        ttk.Button(frame_archivos, text="Seleccionar", command=lambda: self._seleccionar_archivo("previo")).grid(row=1, column=2, padx=5)

        # Archivo nuevo
        tk.Label(frame_archivos, text="Archivo nuevo:").grid(row=2, column=0, sticky="w")
        tk.Entry(frame_archivos, textvariable=self.path_nuevo, width=80).grid(row=2, column=1, padx=5)
        ttk.Button(frame_archivos, text="Seleccionar", command=lambda: self._seleccionar_archivo("nuevo")).grid(row=2, column=2, padx=5)

        ttk.Separator(self.root, orient="horizontal").pack(fill="x", padx=10, pady=5)

        # Hoja y parámetros
        frame_hoja = tk.LabelFrame(self.root, text="Configuración de hoja y estructura", padx=10, pady=10)
        frame_hoja.pack(fill="x", padx=10, pady=5)

        tk.Label(frame_hoja, text="Hoja común:").grid(row=0, column=0, sticky="w")
        self.hoja_combo = ttk.Combobox(frame_hoja, state="readonly", width=50)
        self.hoja_combo.grid(row=0, column=1, padx=5)
        ttk.Button(frame_hoja, text="Cargar hoja", command=self._cargar_hoja).grid(row=0, column=2, padx=5)

        tk.Label(frame_hoja, text="Conceptos (Identifique columna en Excel):").grid(row=1, column=0, sticky="w")
        tk.Entry(frame_hoja, textvariable=self.col_concepto_letra, width=10).grid(row=1, column=1, sticky="w", padx=5)

        tk.Label(frame_hoja, text="Rango de fila conceptos (inicio-fin):").grid(row=1, column=2, sticky="e")
        fila_frame = tk.Frame(frame_hoja)
        fila_frame.grid(row=1, column=3, sticky="w")
        tk.Entry(fila_frame, textvariable=self.fila_concepto_ini, width=6).pack(side="left")
        tk.Label(fila_frame, text=" - ").pack(side="left")
        tk.Entry(fila_frame, textvariable=self.fila_concepto_fin, width=6).pack(side="left")

        tk.Label(frame_hoja, text="Encabezado de meses o años [No. fila:]").grid(row=2, column=0, sticky="w")
        tk.Entry(frame_hoja, textvariable=self.fila_meses, width=10).grid(row=2, column=1, sticky="w", padx=5)

        tk.Label(frame_hoja, text="Rango de columnas de cabecera (inicio-fin):").grid(row=2, column=2, sticky="e")
        meses_frame = tk.Frame(frame_hoja)
        meses_frame.grid(row=2, column=3, sticky="w")
        tk.Entry(meses_frame, textvariable=self.col_meses_ini, width=6).pack(side="left")
        tk.Label(meses_frame, text=" - ").pack(side="left")
        tk.Entry(meses_frame, textvariable=self.col_meses_fin, width=6).pack(side="left")

        tk.Label(frame_hoja, text="Año a comparar (etiqueta del archivo):").grid(row=3, column=0, sticky="w")
        tk.Entry(frame_hoja, textvariable=self.anio_entry, width=10).grid(row=3, column=1, sticky="w", padx=5)

        # Orientación de salida
        tk.Label(frame_hoja, text="Orientación de salida:").grid(row=4, column=0, sticky="w")
        combo_orientacion = ttk.Combobox(frame_hoja, textvariable=self.orientacion, state="readonly",
                     values=["Analisis_horizontal", "Analisis_vertical"])
        combo_orientacion.grid(row=4, column=1, padx=5, sticky="w")
        
        # Reaccionar al cambio de orientación: actualizar lista de selección
        self.orientacion.trace_add("write", self._refrescar_lista_seleccion)

        ttk.Button(frame_hoja, text="Detectar conceptos y cabeceras", command=self._detectar_conceptos_y_meses).grid(row=5, column=2, padx=5)

        ttk.Separator(self.root, orient="horizontal").pack(fill="x", padx=10, pady=5)

        # Selección dinámica (conceptos o años)
        frame_conceptos = tk.LabelFrame(self.root, text="Selección para comparar", padx=10, pady=10)
        frame_conceptos.pack(fill="both", expand=True, padx=10, pady=10)

        ttk.Button(frame_conceptos, text="Seleccionar/Deseleccionar todos",
                   command=self._toggle_todos).pack(pady=5)

        canvas = tk.Canvas(frame_conceptos)
        scrollbar = tk.Scrollbar(frame_conceptos, orient="vertical", command=canvas.yview, width=30)
        self.scrollable_frame = tk.Frame(canvas)

        self.scrollable_frame.bind("<Configure>", lambda e: canvas.configure(scrollregion=canvas.bbox("all")))
        canvas.create_window((0, 0), window=self.scrollable_frame, anchor="nw")
        canvas.configure(yscrollcommand=scrollbar.set)

        canvas.pack(side="left", fill="both", expand=True)
        scrollbar.pack(side="right", fill="y")


    # ------------------------ Lógica ------------------------

    def _seleccionar_archivo(self, tipo):
        path = filedialog.askopenfilename(filetypes=[("Archivos Excel", "*.xlsx")])
        if not path:
            return
        if tipo == "previo":
            self.path_prev.set(path)
        elif tipo == "nuevo":
            self.path_nuevo.set(path)
        elif tipo == "base":
            self.path_base.set(path)
        self._actualizar_hojas_comunes()
    
    def _actualizar_hojas_comunes(self):
        hojas_prev = self._hojas_visibles(self.path_prev.get())
        hojas_nuevo = self._hojas_visibles(self.path_nuevo.get())
        hojas_base = self._hojas_visibles(self.path_base.get())

        # Intersección de hojas visibles entre los tres archivos (si base está seleccionada)
        conjuntos = [set(h) for h in [hojas_prev, hojas_nuevo] if h]
        if hojas_base:
            conjuntos.append(set(hojas_base))

        comunes = sorted(list(set.intersection(*conjuntos))) if conjuntos else []
        self.hoja_combo["values"] = comunes
        if comunes:
            self.hoja_combo.set(comunes[0])
        else:
            self.hoja_combo.set("")
            self.sheet = None

    def _hojas_visibles(self, path):
        if not path:
            return []
        try:
            wb = load_workbook(path, read_only=True, data_only=True)
            return [s.title for s in wb.worksheets if s.sheet_state == "visible"]
        except Exception:
            return []
  
    def _cargar_hoja(self):
        hoja = self.hoja_combo.get().strip()
        if not hoja:
            messagebox.showwarning("Hoja", "Selecciona una hoja común visible en los archivos.")
            return
        self.sheet = hoja
        messagebox.showinfo("Hoja", f"Hoja seleccionada: {hoja}")

    def _detectar_conceptos_y_meses(self):
        try:
            fila_ini = int(self.fila_concepto_ini.get())
            fila_fin = int(self.fila_concepto_fin.get())
            fila_meses = int(self.fila_meses.get())
            col_concepto = letra_a_indice_excel(self.col_concepto_letra.get())
            col_m_ini = letra_a_indice_excel(self.col_meses_ini.get())
            col_m_fin = letra_a_indice_excel(self.col_meses_fin.get())  # corregido
        except Exception as e:
            messagebox.showerror("Parámetros", f"{e}")
            return

        if not self.path_nuevo.get() or not self.path_prev.get():
            messagebox.showwarning("Archivos", "Selecciona los archivos (previo y nuevo).")
            return
        if not self.sheet:
            messagebox.showwarning("Hoja", "Selecciona y carga la hoja común.")
            return

        # Carga de conceptos y cabeceras desde archivo "nuevo"
        try:
            wb = load_workbook(self.path_nuevo.get(), read_only=True, data_only=True)
            ws = wb[self.sheet]
        except Exception as e:
            messagebox.showerror("Lectura", f"No se pudo abrir el archivo nuevo: {e}")
            return

        # Detectar cabeceras (años/meses)
        meses = []
        for c in range(col_m_ini, col_m_fin + 1):
            cell = ws.cell(row=fila_meses, column=c + 1).value
            meses.append(self._leer_valor_cabecera(cell, indice_a_letra_excel(c)))
        self.meses_orden = meses

        # Detectar conceptos (p.ej. meses en primera columna)
        conceptos = []
        for r in range(fila_ini, fila_fin + 1):
            val = ws.cell(row=r, column=col_concepto + 1).value
            if val is None:
                continue
            conceptos.append(str(val).strip())
        conceptos = sorted(list(set(conceptos)))
        self.conceptos_lista = conceptos

        # Refrescar lista según orientación
        self._refrescar_lista_seleccion()

        messagebox.showinfo("Detección", f"Detectados {len(conceptos)} conceptos y {len(meses)} cabeceras (meses/años).")


    def _refrescar_lista_seleccion(self, *args):
        for child in self.scrollable_frame.winfo_children():
            child.destroy()
        self.conceptos_check_vars = {}

        modo = self.orientacion.get()
        items = self.conceptos_lista if modo == "Analisis_horizontal" else self.meses_orden

        for i, item in enumerate(items):
            var = tk.BooleanVar(value=False)
            cb = ttk.Checkbutton(self.scrollable_frame, text=str(item), variable=var,
                                 command=self._limitar_seleccion)
            cb.grid(row=i, column=0, sticky="w", padx=6, pady=2)
            self.conceptos_check_vars[item] = var

    def _toggle_todos(self):
        seleccionados = [c for c, v in self.conceptos_check_vars.items() if v.get()]
        if len(seleccionados) < len(self.conceptos_check_vars):
            count = 0
            for c, v in self.conceptos_check_vars.items():
                if count < self.max_conceptos:
                    v.set(True)
                    count += 1
                else:
                    v.set(False)
        else:
            for v in self.conceptos_check_vars.values():
                v.set(False)

    def _limitar_seleccion(self):
        marcados = [c for c, v in self.conceptos_check_vars.items() if v.get()]
        if len(marcados) > self.max_conceptos:
            for v in self.conceptos_check_vars.values():
                v.set(False)
            for i, c in enumerate(sorted(self.conceptos_check_vars.keys(), key=lambda x: str(x))):
                if i < self.max_conceptos:
                    self.conceptos_check_vars[c].set(True)
            messagebox.showwarning("Límite", f"Máximo {self.max_conceptos} elementos en la selección.")


    def _extraer_tabla(self, path):
        # Devuelve dict: {concepto: {cabecera: valor}} y lista cabeceras detectadas
        try:
            wb = load_workbook(path, read_only=True, data_only=True)
            ws = wb[self.sheet]
        except Exception as e:
            raise RuntimeError(f"No se pudo abrir '{path}': {e}")

        fila_ini = int(self.fila_concepto_ini.get())
        fila_fin = int(self.fila_concepto_fin.get())
        fila_meses = int(self.fila_meses.get())
        col_concepto = letra_a_indice_excel(self.col_concepto_letra.get())
        col_m_ini = letra_a_indice_excel(self.col_meses_ini.get())
        col_m_fin = letra_a_indice_excel(self.col_meses_fin.get())

        meses = self.meses_orden[:]
        if not meses:
            meses = []
            for c in range(col_m_ini, col_m_fin + 1):
                cell = ws.cell(row=fila_meses, column=c + 1).value
                meses.append(self._leer_valor_cabecera(cell, indice_a_letra_excel(c)))

        data = {}
        for r in range(fila_ini, fila_fin + 1):
            concepto_val = ws.cell(row=r, column=col_concepto + 1).value
            if concepto_val is None:
                continue
            concepto = str(concepto_val).strip()
            fila_dict = {}
            for idx_m, c in enumerate(range(col_m_ini, col_m_fin + 1)):
                m = meses[idx_m] if idx_m < len(meses) else indice_a_letra_excel(c)
                val = ws.cell(row=r, column=c + 1).value
                try:
                    num = float(val) if val is not None and str(val).strip() != "" else 0.0
                except Exception:
                    num = 0.0
                fila_dict[m] = num
            data[concepto] = fila_dict
        return data, meses

    def _seleccionar_pestanas(self, lista_pestanas):
        top = tk.Toplevel(self.root)
        top.title("Seleccionar pestañas a generar")
        top.geometry("400x400")
        top.grab_set()

        tk.Label(top, text="Seleccione las pestañas que desea incluir:",
                 font=("Segoe UI", 11, "bold")).pack(pady=10)

        canvas = tk.Canvas(top)
        scrollbar = tk.Scrollbar(top, orient="vertical", command=canvas.yview)
        frame = tk.Frame(canvas)

        frame.bind("<Configure>", lambda e: canvas.configure(scrollregion=canvas.bbox("all")))
        canvas.create_window((0, 0), window=frame, anchor="nw")
        canvas.configure(yscrollcommand=scrollbar.set)

        canvas.pack(side="left", fill="both", expand=True)
        scrollbar.pack(side="right", fill="y")

        vars_dict = {}
        for i, nombre in enumerate(lista_pestanas):
            var = tk.BooleanVar(value=True)
            chk = ttk.Checkbutton(frame, text=str(nombre), variable=var)
            chk.grid(row=i, column=0, sticky="w", padx=5, pady=2)
            vars_dict[nombre] = var

        def confirmar():
            self.pestanas_seleccionadas = [n for n, v in vars_dict.items() if v.get()]
            top.destroy()

        ttk.Button(top, text="Aceptar", command=confirmar).pack(pady=10)
        top.wait_window()


    def _comparar(self):
        if not self.path_prev.get() or not self.path_nuevo.get() or not self.path_base.get():
            messagebox.showwarning("Archivos", "Selecciona los tres archivos: previo, nuevo y año anterior.")
            return
        if not self.sheet:
            messagebox.showwarning("Hoja", "Selecciona y carga la hoja común.")
            return

        seleccionados = [c for c, v in self.conceptos_check_vars.items() if v.get()]
        if not seleccionados:
            messagebox.showwarning("Selección", "Selecciona al menos un elemento.")
            return
        if len(seleccionados) > self.max_conceptos:
            messagebox.showwarning("Selección", f"Máximo {self.max_conceptos} elementos.")
            return

        try:
            data_prev, meses_prev = self._extraer_tabla(self.path_prev.get())
            data_nuevo, meses_nuevo = self._extraer_tabla(self.path_nuevo.get())
            data_base, meses_base = self._extraer_tabla(self.path_base.get())
        except Exception as e:
            messagebox.showerror("Lectura", str(e))
            return

        # Alinear cabeceras (años/meses) comunes entre los tres
        cabeceras_comunes = [m for m in self.meses_orden if m in meses_prev and m in meses_nuevo and m in meses_base]
        if not cabeceras_comunes:
            # fallback si no hay orden cargado: intersección de las tres listas detectadas
            cabeceras_comunes = [m for m in meses_prev if m in meses_nuevo and m in meses_base]
            if not cabeceras_comunes:
                messagebox.showerror("Cabeceras", "No hay cabeceras comunes entre los tres archivos.")
                return

        sugerido = f"Comparativo_{self.anio_entry.get()}_{datetime.now().strftime('%Y%m%d_%H%M')}.xlsx"
        save_path = filedialog.asksaveasfilename(
            defaultextension=".xlsx",
            filetypes=[("Excel", "*.xlsx")],
            initialfile=sugerido,
            title="Guardar comparativo"
        )
        if not save_path:
            return

        wb_out = Workbook()
        default_ws = wb_out.active
        wb_out.remove(default_ws)

        modo = self.orientacion.get()

        # Mostrar ventana para elegir pestañas
        if modo == "Analisis_horizontal":
            lista_pestanas = cabeceras_comunes
        else:
            lista_pestanas = sorted(self.conceptos_lista)

        self._seleccionar_pestanas(lista_pestanas)
        if not self.pestanas_seleccionadas:
            messagebox.showwarning("Selección", "No seleccionó ninguna pestaña.")
            return

        #Creacion de hojas seleccionadas
        if modo == "Analisis_horizontal":
            conceptos_sel = seleccionados
            for cab in self.pestanas_seleccionadas:
                ws_out = wb_out.create_sheet(title=safe_sheet_name(cab))

                headers = [
                    "Concepto", "Previo", "Nuevo", "Base",
                    "Dif_abs",
                    "Tasa_var_prev", "Tasa_var_nuevo", "Dif_tasas"
                ]
                for col_idx, h in enumerate(headers, start=1):
                    cell = ws_out.cell(row=1, column=col_idx, value=h)
                    cell.font = Font(bold=True, color="FFFFFF")
                    cell.fill = PatternFill("solid", fgColor="1F4E79")
                    cell.alignment = Alignment(horizontal="center")

                r = 2
                for concepto in conceptos_sel:
                    prev_val = data_prev.get(concepto, {}).get(cab, 0.0)
                    nuevo_val = data_nuevo.get(concepto, {}).get(cab, 0.0)
                    base_val = data_base.get(concepto, {}).get(cab, 0.0)
                    dif_abs = nuevo_val - prev_val

                    tasa_prev = ((prev_val - base_val) / base_val * 100.0) if base_val != 0 else None
                    tasa_nuevo = ((nuevo_val - base_val) / base_val * 100.0) if base_val != 0 else None
                    dif_tasas = (tasa_nuevo - tasa_prev) if (tasa_prev is not None and tasa_nuevo is not None) else None

                    ws_out.cell(row=r, column=1, value=concepto)
                    ws_out.cell(row=r, column=2, value=prev_val)
                    ws_out.cell(row=r, column=3, value=nuevo_val)
                    ws_out.cell(row=r, column=4, value=base_val)
                    ws_out.cell(row=r, column=5, value=dif_abs)
                    ws_out.cell(row=r, column=6, value=tasa_prev if tasa_prev is not None else "")
                    ws_out.cell(row=r, column=7, value=tasa_nuevo if tasa_nuevo is not None else "")
                    ws_out.cell(row=r, column=8, value=dif_tasas if dif_tasas is not None else "")
                    r += 1

                widths = [28, 14, 14, 14, 14, 18, 18, 18]
                for i, w in enumerate(widths, start=1):
                    ws_out.column_dimensions[get_column_letter(i)].width = w
                for rr in range(2, r):
                    ws_out.cell(row=rr, column=2).number_format = "#,##0.00"
                    ws_out.cell(row=rr, column=3).number_format = "#,##0.00"
                    ws_out.cell(row=rr, column=4).number_format = "#,##0.00"
                    ws_out.cell(row=rr, column=5).number_format = "#,##0.00"
                    ws_out.cell(row=rr, column=6).number_format = "0.00"
                    ws_out.cell(row=rr, column=7).number_format = "0.00"
                    ws_out.cell(row=rr, column=8).number_format = "0.00"
        else:
            # Selección son cabeceras (años/meses); hojas por concepto
            cabeceras_sel = seleccionados
            for concepto in self.pestanas_seleccionadas:
                ws_out = wb_out.create_sheet(title=safe_sheet_name(concepto))

                headers = [
                    "Año/Mes", "Previo", "Nuevo", "Base",
                    "Dif_abs",
                    "Tasa_var_prev", "Tasa_var_nuevo", "Dif_tasas"
                ]
                for col_idx, h in enumerate(headers, start=1):
                    cell = ws_out.cell(row=1, column=col_idx, value=h)
                    cell.font = Font(bold=True, color="FFFFFF")
                    cell.fill = PatternFill("solid", fgColor="1F4E79")
                    cell.alignment = Alignment(horizontal="center")

                r = 2
                for cab in cabeceras_sel:
                    prev_val = data_prev.get(concepto, {}).get(cab, 0.0)
                    nuevo_val = data_nuevo.get(concepto, {}).get(cab, 0.0)
                    base_val = data_base.get(concepto, {}).get(cab, 0.0)
                    dif_abs = nuevo_val - prev_val

                    tasa_prev = ((prev_val - base_val) / base_val * 100.0) if base_val != 0 else None
                    tasa_nuevo = ((nuevo_val - base_val) / base_val * 100.0) if base_val != 0 else None
                    dif_tasas = (tasa_nuevo - tasa_prev) if (tasa_prev is not None and tasa_nuevo is not None) else None

                    ws_out.cell(row=r, column=1, value=cab)
                    ws_out.cell(row=r, column=2, value=prev_val)
                    ws_out.cell(row=r, column=3, value=nuevo_val)
                    ws_out.cell(row=r, column=4, value=base_val)
                    ws_out.cell(row=r, column=5, value=dif_abs)
                    ws_out.cell(row=r, column=6, value=tasa_prev if tasa_prev is not None else "")
                    ws_out.cell(row=r, column=7, value=tasa_nuevo if tasa_nuevo is not None else "")
                    ws_out.cell(row=r, column=8, value=dif_tasas if dif_tasas is not None else "")
                    r += 1

                widths = [18, 14, 14, 14, 14, 18, 18, 18]
                for i, w in enumerate(widths, start=1):
                    ws_out.column_dimensions[get_column_letter(i)].width = w
                for rr in range(2, r):
                    ws_out.cell(row=rr, column=2).number_format = "#,##0.00"
                    ws_out.cell(row=rr, column=3).number_format = "#,##0.00"
                    ws_out.cell(row=rr, column=4).number_format = "#,##0.00"
                    ws_out.cell(row=rr, column=5).number_format = "#,##0.00"
                    ws_out.cell(row=rr, column=6).number_format = "0.00"
                    ws_out.cell(row=rr, column=7).number_format = "0.00"
                    ws_out.cell(row=rr, column=8).number_format = "0.00"

        try:
            wb_out.save(save_path)
        except Exception as e:
            messagebox.showerror("Guardar", f"No se pudo guardar el archivo: {e}")
            return

        messagebox.showinfo("Éxito", f"Comparativo guardado en:\n{save_path}")

# Lanzador
if __name__ == "__main__":
    root = tk.Tk()
    app = ComparadorExcel(root)
    root.mainloop()


Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\carlo\miniconda3\Lib\tkinter\__init__.py", line 2068, in __call__
    return self.func(*args)
           ~~~~~~~~~^^^^^^^
  File "C:\Users\carlo\AppData\Local\Temp\ipykernel_9568\95280993.py", line 420, in _comparar
    self._seleccionar_pestanas(lista_pestanas)
    ^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: 'ComparadorExcel' object has no attribute '_seleccionar_pestanas'
